---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

* 워크플로우
  * `HubClient() 객체` 생성 → `client.push()` 사용

---

In [ ]:
# 1_새 프롬프트 생성하기
from langsmith import Client
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langsmith import Client

import os
import json


# 클라이언트 생성 
api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=api_key)

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langsmith import traceable

# LLM 초기화
gemini_lc = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0.7,                                    
        max_output_tokens=4096,
    )

---

#### **2) Map Prompt**

* 역할
  * **여러 문서(`GIVEN DOCUMENTS`)** 를 받아서 **중요한 주제** 와 **핵심 아이디어** 를 **번호** 가 붙은 **목록** 으로 추출

* 특징
  * **최대 5가지 핵심 주제** 까지만 뽑아서 과도한 정보를 줄임
  * 원문 내 주요 **내용과 사례** 를 포함해 전체 내용을 요약하되, 쉽고 명확하게 만듦

In [4]:
# Map Prompt

# 프롬프트의 제목
# 특별한 기능 X, 프롬프트 식별하기 위해 사용 (문서의 내용을 지도처럼 요약한다는 의미)
prompt_title = "map-prompt"

# 프롬프트의 본문
# 역할 설정 + 요청 사항 + 요약 조건 + 포함 및 제외 내용 + 요약의 길이 
# GIVEN DOCUMENTS = 모델이 요약해야 할 문서의 입력 섹션
# {docs} = 실제 요약할 문서 내용이 들어갈 {placeholder}
# FORMAT = 출력 양식의 예시
# CAUTION = 주의사항 = 제한 사항 (5개 이상의 main theme 나열 X)
# Helpful Answer = 모델이 답변을 시작하는 지점을 나타내는 가이드 라인 
prompt_template = """You are a helpful expert journalist in extracting the main themes from a GIVEN DOCUMENTS below.
Please provide a comprehensive summary of the GIVEN DOCUMENTS in numbered list format. 
The summary should cover all the key points and main ideas presented in the original text, while also condensing the information into a concise and easy-to-understand format. 
Please ensure that the summary includes relevant details and examples that support the main ideas, while avoiding any unnecessary information or repetition. 
The length of the summary should be appropriate for the length and complexity of the original text, providing a clear and accurate overview without omitting any important information.

GIVEN DOCUMENTS:
{docs}

FORMAT:
1. main theme 1
2. main theme 2
3. main theme 3
...

CAUTION:
- DO NOT list more than 5 main themes.

Helpful Answer:
"""

prompt_map = PromptTemplate.from_template(prompt_template)

In [ ]:
print(type(prompt_map))                                 # <class 'langchain_core.prompts.prompt.PromptTemplate'>

In [ ]:
# LangSmith 프롬프트 등록하기
# 프롬프트 업로드 (push_prompt 사용)
result_map = client.push_prompt("prompt_template_map", object=prompt_map)
print('\n', "-" * 50, '\n')
print("✅ 프롬프트 업로드 결과:", result_map)
print('\n', "-" * 50)

# 업로드한 허브에서 가져오기
prompt_map_from_hub = client.pull_prompt("prompt_template_map")
print('\n', "-" * 50, '\n')
print("허브에서 프롬프트를 성공적으로 가져왔습니다.")
print('\n', "-" * 50)


# 5_LangSmith Client로 최근 커밋 해시 조회하기
prompt_id = "prompt_template_map"

# 1) 프롬프트 정보 가져오기
prompt_info_map = client.get_prompt(prompt_id)

print("프롬프트 정보:", prompt_info_map)

prompt_info_map = client.get_prompt(prompt_id)

print("프롬프트 정보:")
for key, value in prompt_info_map.__dict__.items():
    print(f"{key}: {value}")

# 줄 바꿈
print("\n" + "="*50 + "\n")


# 2) 프롬프트 가져오기
prompt_map_from_hub = client.pull_prompt(prompt_id)
print("프롬프트 객체 타입:", type(prompt_map_from_hub))

# 줄 바꿈
print("\n" + "="*50 + "\n")

# 3) 프롬프트 메타데이터 가져오기 - 만약 metadata 속성이 있으면 출력
if hasattr(prompt_map_from_hub, "metadata"):
    metadata = prompt_map_from_hub.metadata
    print("메타데이터:")
    for key, value in metadata.items():
        print(f" {key}: {value}")
    print("\nJSON 포맷으로 보기:")
    import json
    print(json.dumps(metadata, indent=2, ensure_ascii=False))
else:
    print("메타데이터 속성이 없습니다.")

<small>

* 셀 출력

    ```<plaintext>
    -------------------------------------------------- 

    ✅ 프롬프트 업로드 결과: https://smith.langchain.com/prompts/prompt_template_map/28669aaf?organizationId=2c3342d3-1170-4ffa-86fd-f621199e0b9c

    --------------------------------------------------

    -------------------------------------------------- 

    허브에서 프롬프트를 성공적으로 가져왔습니다.

    --------------------------------------------------
    프롬프트 정보: repo_handle='prompt_template_map' description='' readme='' id='3f2739a0-6fb5-43ee-845b-7ff9eef0b049' tenant_id='2c3342d3-1170-4ffa-86fd-f621199e0b9c' created_at=datetime.datetime(2025, 8, 13, 13, 14, 31, 137986) updated_at=datetime.datetime(2025, 8, 13, 13, 14, 35, 352488) is_public=False is_archived=False tags=['StringPromptTemplate'] original_repo_id=None upstream_repo_id=None owner=None full_name='prompt_template_map' num_likes=0 num_downloads=1 num_views=0 liked_by_auth_user=False last_commit_hash='28669aaffe2816053669e3cba9f2c3751cc1160dd8139ad0b8bf0ad2eb0428bb' num_commits=1 original_repo_full_name=None upstream_repo_full_name=None
    프롬프트 정보:
    repo_handle: prompt_template_map
    description: 
    readme: 
    id: 3f2739a0-6fb5-43ee-845b-7ff9eef0b049
    tenant_id: 2c3342d3-1170-4ffa-86fd-f621199e0b9c
    created_at: 2025-08-13 13:14:31.137986
    updated_at: 2025-08-13 13:14:35.352488
    is_public: False
    is_archived: False
    tags: ['StringPromptTemplate']
    original_repo_id: None
    upstream_repo_id: None
    owner: None
    full_name: prompt_template_map
    num_likes: 0
    num_downloads: 1
    num_views: 0
    liked_by_auth_user: False
    last_commit_hash: 28669aaffe2816053669e3cba9f2c3751cc1160dd8139ad0b8bf0ad2eb0428bb
    num_commits: 1
    original_repo_full_name: None
    upstream_repo_full_name: None

    ==================================================

    프롬프트 객체 타입: <class 'langchain_core.prompts.prompt.PromptTemplate'>

    ==================================================

    메타데이터:
    lc_hub_owner: -
    lc_hub_repo: prompt_template_map
    lc_hub_commit_hash: 28669aaffe2816053669e3cba9f2c3751cc1160dd8139ad0b8bf0ad2eb0428bb

    JSON 포맷으로 보기:
    {
    "lc_hub_owner": "-",
    "lc_hub_repo": "prompt_template_map",
    "lc_hub_commit_hash": "28669aaffe2816053669e3cba9f2c3751cc1160dd8139ad0b8bf0ad2eb0428bb"
    }
    ```
    

In [ ]:
# gemini로 LLM 초기화 및 Chain 생성하기

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langsmith import traceable

# LLM 초기화
gemini_lc = gemini_lc               # 위에서 정의함

# chain_map 구성
chain_map = prompt_map | gemini_lc | StrOutputParser()

# context_text 입력하기
context_text = """
최근 서울시는 인공지능 기반 교통 신호 제어 시스템을 시범 도입했습니다.
이 시스템은 도로 상황과 교통량을 실시간 분석하여 신호를 조절해 교통 체증을 줄입니다.
관계자는 향후 빅데이터와 결합해 더욱 정교한 예측과 제어가 가능할 것이라고 밝혔습니다.
하지만 개인정보 보호와 시스템 오작동에 대한 우려도 제기되고 있습니다.
"""
result = chain_map.invoke({"docs": context_text})
print(result)

<small>

* 셀 출력 (temperature=0.7, max_tokens=4,096)(1.8s)

    ```<plaintext>
    Here's a summary of the provided document in a numbered list format:

    1.  **Introduction of AI-based Traffic Signal Control System:** Seoul City has recently piloted an artificial intelligence-based traffic signal control system.
    2.  **Functionality and Goal:** This system analyzes real-time road conditions and traffic volume to adjust signals, aiming to reduce traffic congestion.
    3.  **Future Potential:** Officials state that combining this system with big data will enable more sophisticated prediction and control in the future.
    4.  **Concerns Raised:** Despite the benefits, concerns have been raised regarding personal information protection and potential system malfunctions.
    ```

In [ ]:
# map_chain 한국어 버전으로 프롬프트 만들어보기 

prompt_title = "map-prompt-ko"

prompt_template = """주어진 문서에서 핵심 주제를 추출하는 데 능숙한 유능한 전문 기자 역할을 맡아주세요.
아래에 제시된 문서를 한국어로 번호가 매겨진 목록 형식으로 포괄적으로 요약해 주세요.
요약은 원문의 모든 핵심 내용과 주요 아이디어를 다루면서도, 간결하고 이해하기 쉬운 형식으로 정보를 압축해야 합니다.
요약에 주요 아이디어를 뒷받침하는 관련 세부 정보와 예시를 포함하되, 불필요한 정보나 반복적인 내용은 피해주세요.
요약의 길이는 원문의 길이와 복잡성에 적합해야 하며, 중요한 정보를 누락하지 않으면서 명확하고 정확한 개요를 제공해야 합니다.

주어진 문서:
{docs}

형식:
1. 주요 주제 1
2. 주요 주제 2
3. 주요 주제 3
...

주의사항:
- 반드시 한국어로 작성하세요. 
- 5개 이상의 주요 주제를 나열하지 마세요.

유용한 답변:
"""

prompt_map_ko = PromptTemplate.from_template(prompt_template)
print(type(prompt_map_ko))                                       # <class 'langchain_core.prompts.prompt.PromptTemplate'>

In [ ]:
# LangSmith 프롬프트 등록하기
# 프롬프트 업로드 (push_prompt 사용)
result_map_ko = client.push_prompt("prompt_template_map_ko", object=prompt_map_ko)
print('\n', "-" * 50, '\n')
print("✅ 프롬프트 업로드 결과:", result_map_ko)
print('\n', "-" * 50)

# 업로드한 허브에서 가져오기
prompt_map_ko_from_hub = client.pull_prompt("prompt_template_map_ko")
print('\n', "-" * 50, '\n')
print("허브에서 프롬프트를 성공적으로 가져왔습니다.")
print('\n', "-" * 50)


# LangSmith Client로 최근 커밋 해시 조회하기
prompt_id = "prompt_template_map_ko"
prompt_info_map_ko = client.get_prompt(prompt_id)

print("프롬프트 정보:", prompt_info_map)

prompt_info_map_ko = client.get_prompt(prompt_id)

print("프롬프트 정보:")
for key, value in prompt_info_map_ko.__dict__.items():
    print(f"{key}: {value}")

# 줄 바꿈
print("\n" + "="*50 + "\n")


# 2) 프롬프트 가져오기
prompt_map_ko_from_hub = client.pull_prompt(prompt_id)
print("프롬프트 객체 타입:", type(prompt_map_ko_from_hub))

# 줄 바꿈
print("\n" + "="*50 + "\n")

# 3) 프롬프트 메타데이터 가져오기 - 만약 metadata 속성이 있으면 출력
if hasattr(prompt_map_ko_from_hub, "metadata"):
    metadata = prompt_map_ko_from_hub.metadata
    print("메타데이터:")
    for key, value in metadata.items():
        print(f" {key}: {value}")
    print("\nJSON 포맷으로 보기:")
    import json
    print(json.dumps(metadata, indent=2, ensure_ascii=False))
else:
    print("메타데이터 속성이 없습니다.")

<small>

* 셀 출력 

    ```<plaintext>
    -------------------------------------------------- 

    ✅ 프롬프트 업로드 결과: https://smith.langchain.com/prompts/prompt_template_map_ko/6c86a411?organizationId=2c3342d3-1170-4ffa-86fd-f621199e0b9c

    --------------------------------------------------

    -------------------------------------------------- 

    허브에서 프롬프트를 성공적으로 가져왔습니다.

    --------------------------------------------------
    프롬프트 정보: repo_handle='prompt_template_map' description='' readme='' id='3f2739a0-6fb5-43ee-845b-7ff9eef0b049' tenant_id='2c3342d3-1170-4ffa-86fd-f621199e0b9c' created_at=datetime.datetime(2025, 8, 13, 13, 14, 31, 137986) updated_at=datetime.datetime(2025, 8, 13, 13, 14, 35, 352488) is_public=False is_archived=False tags=['StringPromptTemplate'] original_repo_id=None upstream_repo_id=None owner=None full_name='prompt_template_map' num_likes=0 num_downloads=1 num_views=0 liked_by_auth_user=False last_commit_hash='28669aaffe2816053669e3cba9f2c3751cc1160dd8139ad0b8bf0ad2eb0428bb' num_commits=1 original_repo_full_name=None upstream_repo_full_name=None
    프롬프트 정보:
    repo_handle: prompt_template_map_ko
    description: 
    readme: 
    id: 2c41fc5e-098d-4028-bdf0-78dd60803f09
    tenant_id: 2c3342d3-1170-4ffa-86fd-f621199e0b9c
    created_at: 2025-08-13 13:54:21.160235
    updated_at: 2025-08-13 13:54:22.538614
    is_public: False
    is_archived: False
    tags: ['StringPromptTemplate']
    original_repo_id: None
    upstream_repo_id: None
    owner: None
    full_name: prompt_template_map_ko
    num_likes: 0
    num_downloads: 1
    num_views: 0
    liked_by_auth_user: False
    last_commit_hash: 6c86a41142b1753cf4ffbb4c7a395fbda931567369fba5de4cb2876d22a72432
    num_commits: 1
    original_repo_full_name: None
    upstream_repo_full_name: None

    ==================================================

    프롬프트 객체 타입: <class 'langchain_core.prompts.prompt.PromptTemplate'>

    ==================================================

    메타데이터:
    lc_hub_owner: -
    lc_hub_repo: prompt_template_map_ko
    lc_hub_commit_hash: 6c86a41142b1753cf4ffbb4c7a395fbda931567369fba5de4cb2876d22a72432

    JSON 포맷으로 보기:
    {
    "lc_hub_owner": "-",
    "lc_hub_repo": "prompt_template_map_ko",
    "lc_hub_commit_hash": "6c86a41142b1753cf4ffbb4c7a395fbda931567369fba5de4cb2876d22a72432"
    }

    ```

In [ ]:
# gemini로 LLM 초기화 및 Chain 생성하기

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langsmith import traceable

# LLM 초기화
gemini_lc = gemini_lc                                               # 위에서 정의함

# chain_map 구성
chain_map_ko = prompt_map_ko | gemini_lc | StrOutputParser()

# context_text 입력하기
context_text = """
오픈AI가 차세대 모델 'GPT-5'를 출시했다. 'GPT-4' 등장 당시처럼 다른 모델과 큰 성능 차를 보이지는 못했으나, 코딩과 에이전트 기능 향상으로 사용성을 넓혔으며, 특히 환각이 크게 줄어든 것으로 나타났다. 단순 챗봇을 넘어 AI 에이전트로의 진화를 예고했다.

오픈AI는 7일(현지시간) 라이브 스트리밍 행사를 통해 GPT-5를 공개했다.

이날부터 모든 챗GPT 무료 사용자에게 기본 모델로 제공된다. 또 월 200달러의 챗GPT 프로 사용자는 추가 컴퓨팅 리소스를 활용해 더 나은 답변을 생성하는 'GPT-5 프로' 버전도 이용할 수 있다. 팀과 에듀, 엔터프라이즈 구독자는 다음 주부터 기본 모델로 사용할 수 있게 된다.

스트리밍에 앞서 오픈AI는 미디어 브리핑을 열고 별도의 모델 설명에 나섰다. 샘 알트먼 CEO는 기존 o 시리즈 모델의 추론 능력과 GPT 시리즈의 빠른 응답 속도를 결합한 모델이라고 설명했다.

그는 "GPT-5는 세계 최고의 모델"이라며 "경제적으로 가장 가치 있는 작업, 즉 인공일반지능(AGI)에서 인간보다 더 나은 성과를 낼 수 있는 AI를 개발하려는 회사의 여정에서 중요한 진전을 나타낸다"라고 말했다. 또 "이런 모델이 등장할 것이라고 이전에는 상상할 수 없었다"라고 강조했다.

특히 GPT-4가 AI 챗봇으로 답변에 초점을 맞췄다면, GPT-5는 사용자를 대신해 소프트웨어를 생성하고 연구 보고서를 작성하는 등 다양한 작업을 수행하는 것이 핵심이라고 밝혔다.

또 실시간 라우터를 탑재, 사용자 쿼리에 맞춰 자동으로 추론이나 비추론, 도구 사용 모드 등을 자동 선택한다고 전했다. 이제부터는 사용자가 모델을 고르지 않아도 된다.

닉 털리 오픈AI 부사장은 출시와 동시에 무료 개방을 결정한 데 대해 "우리가 사명을 실천하는 방법 중 하나로, 실제로 사람들에게 이익이 되도록 하는 것"이라고 설명했다. 오픈AI는 이제까지 첨단 모델을 유료 사용자에게 우선 공개한 뒤 일정 시간이 지난 뒤 무료로 확대하는 방식을 채택해 왔다.



이날 미디어 브리핑은 모델 출시에 앞서 열린 이례적인 행사다. 또 오픈AI는 GPT-5 출시를 앞두고 비공식적인 홍보 활동을 펼쳐 왔다.

이에 따라 벤치마크에서 특별한 차이를 보여주지 못하는 것이 아니냐는 분석이 나왔고, 이는 이날 공개한 수치로 확인됐다. 벤치마크 상 최고의 모델이 맞기는 하지만, 격차가 크지 않고 일부에서는 다른 모델에 뒤졌다.

알트만 CEO가 '바이브 코딩'에 탁월하다고 밝히는 등 오픈AI는 가장 먼저 GPT-5의 코딩 성능을 강조했다.

코딩 능력 테스트인 'SWE-벤치 베리파이드'에서 GPT-5는 첫 시도에서 74.9%의 점수를 기록했다. 이는 구글의 '제미나이 2.5 프로(59.6%)'를 크게 앞섰지만, 앤트로픽의 '클로드 오퍼스 4.1(74.5%)'는 간신히 앞질렀다.

오픈AI는 에이전트 작업에 탁월하다는 점도 강조했지만, 모델의 온라인 작업 수행 능력을 측정하는 '타우벤치(Tau-bench)'에서는 다른 모델에 떨어졌다. 항공사 웹사이트 탐색 능력을 측정하는 테스트에서 63.5%를 기록, 64.8%를 기록한 o3보다 약간 낮은 성적을 보였다. 소매 웹사이트 탐색 능력을 측정하는 다른 테스트에서는 81.1%를 기록, 클로드 오퍼스 4.1(82.4%)보다 낮았다.

수학, 인문학, 자연과학 전반에 걸쳐 AI 모델 성능을 측정하는 '인류의 마지막 시험(HLE)'에서 확장 추론 기능을 탑재한 GPT-5 프로 버전은 도구를 사용했을 때 42%의 점수를 기록했다. 이는 '그록 4 헤비'의 44.4%보다 약간 낮은 수치다.

박사 수준의 과학 문제를 테스트하는 'GPQA 다이아몬드'에서 GPT-5 프로는 첫 시도에서 89.4%를 기록, 클로드 오퍼스 4.1(80.9%)과 그록 4 헤비(88.9%)를 앞질렀다.


환각률 테스트 결과 
대신, GPT-5의 환각은 크게 줄어든 것으로 나타났다. 의료 주제에 대한 테스트인 '헬스벤치 하드 할루시네이션(HealthBench Hard Hallucinations)'에서 GPT-5는 환각을 기록한 비율이 1.6%에 불과했다.

이는 이전 모델인 GPT-4o와 o3 모델의 12.9%와 15.8%를 크게 낮춘 것이다. 오픈AI는 최근 많은 사람들이 챗GPT를 건강 상담 등에 활용한다는 점에서 중요하며, 이 분야를 강화하겠다고 밝혔다.

또 전반적인 프롬프트에서 환각이 발생한 비율은 4.8%였다. 이 또한 각각 22%와 20.6%의 환각률을 기록한 o3와 GPT-4o보다 상당히 낮은 수치다.

오픈 AI는 앞서 o3와 o4-미니 등 최신 추론 모델이 성능 향상에도 불구, 환각 현상이 악화되는 현상을 발견했다. 이에 대한 원인을 제대로 이해하지 못했다고도 밝혔다.

이 문제를 최근 국제수학올림피아드에서 금메달을 따는 데 활용했던 '범용 검증기(Universial Verifier)'가 해결한 것으로 보인다. 이 기술은 모델 응답을 다른 모델이 검색과 추론 등으로 검토, 수정을 유도하는 방식이다. 추론을 잇는 차세대 AI 기술로 꼽히고 있다.


모델의 기만 행위 테스트 결과 (사진=오픈AI)
또 오픈AI는 창의적 디자인이나 글쓰기처럼 주관적인 영역에서 크게 개선됐다고 밝혔다. 털리 부사장은 "다른 AI 모델보다 더 자연스럽게 반응하고 더 나은 취향을 보인다"라고 말했다.

더불어 이제 챗GPT 설정에서 ▲냉소주의자(Cynic) ▲로봇(Robot) ▲청취자(Listener) ▲괴짜(Nerd) 등 네가지 성격을 선택할 수 있다. 이를 통해 사용자가 특정 방식으로 응답하도록 모델에 직접 요청하지 않아도 된다고 설명했다.

안전 문제도 강조했다. 최근 앤트로픽 연구 결과에 따르면, 첨단 모델은 자신을 방어하기 위해 인간을 협박하거나 거짓말을 하는 경향을 보였다. 그러나 GPT-5는 다른 모델보다 기만적인 행동 비율이 낮다고 전했다.

또 악의적인 행위자와 무해한 요청을 하는 사용자를 구분하는 데 더 뛰어나다고 밝혔다. 따라서 무해한 정보를 요청하는 사용자에게 거부하는 횟수를 줄이고, 동시에 안전하지 않은 질문을 거부할 수 있다는 것이다.

개발자를 위해 ▲GPT-5 ▲GPT-5-미니 ▲GPT-5-나노 등 3가지 모델이 API에 추가됐다. 이를 통해 모델 추론 시간을 조정하고 응답 길이를 제어할 수 있다.

API 가격은 100만 입력 토큰당 1.25달러, 출력 토큰당 10달러다. 이는 GPT-4o의 2.5/10달러, o3의 2/8달러와 비슷한 수준이다.

이처럼 GPT-5는 성능 면에서는 큰 진전을 이루는 데 실패한 것으로 보인다. 그러나 챗봇을 인간을 돕는 도구로 업그레이드했다는 점, 그리고 가장 큰 문제였던 환각을 크게 줄였다는 점에서 주목된다.

특히 이를 실제 환경에서 이를 어떻게 활용하고 어떤 결과를 낼지가 가장 큰 관심사로 떠올랐다.


"""
result = chain_map_ko.invoke({"docs": context_text})
print(result)

<small>

* 셀 출력(temperature=0.7, max_tokens=4,096)(2.9s)

    ```<plaintext>
    Here are the main themes from the provided document about OpenAI's GPT-5:

    1.  **GPT-5 Launch and Accessibility:** OpenAI has launched GPT-5, making it available as the default model for all free ChatGPT users starting immediately. Paid ChatGPT Pro users will have access to a "GPT-5 Pro" version with enhanced computing resources, and team/enterprise subscribers will get it next week.
    2.  **Evolution Beyond Chatbots to AI Agents:** GPT-5 is positioned as an evolution from a simple chatbot to a more capable AI agent. Its core functionality is to perform diverse tasks on behalf of users, such as generating software and writing research reports, moving beyond just providing answers.
    3.  **Improved Coding and Reduced Hallucinations:** GPT-5 shows significant improvements in coding capabilities, outperforming Google's Gemini 2.5 Pro in benchmarks, though closely matching Anthropic's Claude Opus 4.1. A key advancement is a substantial reduction in hallucinations, with a reported 1.6% hallucination rate on medical topics and 4.8% overall, a significant decrease from previous models like GPT-4o.
    4.  **Mixed Benchmark Performance and New Capabilities:** While GPT-5 excels in reducing hallucinations and shows strengths in coding, its performance in agent tasks and some academic benchmarks is mixed, sometimes trailing behind other leading models. New features include a real-time router that automatically selects inference modes and the ability to choose distinct personas for the chatbot (Cynic, Robot, Listener, Nerd).
    5.  **Emphasis on Safety and Developer Options:** OpenAI highlights GPT-5's reduced tendency for deceptive behavior compared to other advanced models and its improved ability to distinguish between malicious and harmless users. For developers, three versions (GPT-5, GPT-5-mini, GPT-5-nano) are available via API with adjustable inference times and response lengths, priced similarly to GPT-4o.
    ```

<small>

* 위는 영어 버전
* 아래는 한국어 버전

<small>

* 셀 출력 (temperature=0.7, max_tokens=4,096)(4.0s)

    ```<plaintext>
    오픈AI의 차세대 모델 GPT-5에 대한 핵심 내용을 다음과 같이 요약합니다.

    1.  **GPT-5 출시 및 주요 특징:** 오픈AI는 차세대 모델 GPT-5를 공개했으며, 이는 이전 모델 대비 코딩 및 에이전트 기능이 향상되었고 환각 현상이 크게 줄어든 것이 특징입니다. 단순 챗봇을 넘어 AI 에이전트로의 진화를 예고하며, 샘 알트먼 CEO는 GPT-5를 "세계 최고의 모델"이라 칭하며 인공일반지능(AGI) 개발에 중요한 진전이라고 강조했습니다.

    2.  **사용자 접근성 및 버전:** GPT-5는 모든 챗GPT 무료 사용자에게 기본 모델로 제공되며, 챗GPT 프로 유료 사용자는 추가 컴퓨팅 리소스를 활용하는 'GPT-5 프로' 버전을 이용할 수 있습니다. 팀, 에듀, 엔터프라이즈 구독자는 다음 주부터 기본 모델로 사용 가능합니다. 이는 기존에 첨단 모델을 유료 사용자에게 우선 공개했던 방식과는 달리, 사명 실천의 일환으로 무료 사용자에게도 즉시 개방하는 결정입니다.

    3.  **성능 벤치마크 및 개선점:** GPT-5는 코딩 능력 테스트에서 구글의 제미나이 2.5 프로를 크게 앞섰지만, 앤트로픽의 클로드 오퍼스 4.1과는 비슷한 수준을 보였습니다. 에이전트 작업에서는 일부 온라인 작업 수행 능력 측정에서 다른 모델에 약간 뒤지는 모습을 보이기도 했습니다. 하지만 가장 큰 성과는 환각 현상 감소로, 의료 주제 테스트에서 GPT-5의 환각 비율은 1.6%로 이전 모델 대비 현저히 낮아졌습니다.

    4.  **AI 에이전트 기능 및 사용자 경험 향상:** GPT-5는 사용자를 대신해 소프트웨어를 생성하거나 연구 보고서를 작성하는 등 다양한 작업을 수행하는 데 중점을 두었습니다. 실시간 라우터 탑재로 사용자 쿼리에 맞춰 추론, 비추론, 도구 사용 모드를 자동으로 선택하며, 사용자가 모델을 선택할 필요가 없어졌습니다. 또한, 냉소주의자, 로봇, 청취자, 괴짜 등 네 가지 성격을 선택하여 모델의 응답 방식을 조절할 수 있게 되었습니다.

    5.  **안전성 및 개발자 지원:** GPT-5는 다른 모델에 비해 기만적인 행동 비율이 낮으며, 악의적인 행위자와 무해한 요청을 구분하는 능력이 향상되었습니다. 이를 통해 무해한 정보를 요청하는 사용자에게 거부하는 횟수를 줄이면서도 안전하지 않은 질문은 거부할 수 있습니다. 개발자를 위해서는 GPT-5, GPT-5-미니, GPT-5-나노 세 가지 모델이 API에 추가되었으며, API 가격은 GPT-4o와 유사한 수준으로 책정되었습니다.
    ```

---

#### **3) Reduce Prompt**

* 역할
  * **여러 개의 요약 리스트** 를 받아서 **가장 중요한 인사이트 10개** 를 뽑고, 이걸 다시 **한국어로 깔끔하게 요약** 해 줍니다.

* 특징
  * 여러 요약을 하나로 압축, 핵심 인사이트만 정리
  * 한국어로 요약해서 현지화

In [ ]:
# 프롬프트 생성

prompt_title = "reduce-prompt"

prompt_template = """You are a helpful expert in summary writing.
You are given numbered lists of summaries.
Extract top 10 most important insights from the summaries.
Then, write a summary of the insights in KOREAN.

LIST OF SUMMARIES:
{doc_summaries}

Helpful Answer:
"""

prompt_reduce = PromptTemplate.from_template(prompt_template)
print(type(prompt_reduce))                                          # <class 'langchain_core.prompts.prompt.PromptTemplate'>

In [ ]:
# 허브에 올리기 
result_reduce = client.push_prompt("prompt_template_reduce", object=prompt_reduce)
print('\n', "-" * 50, '\n')
print("✅ 프롬프트 업로드 결과:", result_reduce)
print('\n', "-" * 50)

# 허브에서 가져오기 
prompt_reduce_from_hub = client.pull_prompt("prompt_template_reduce")
print('\n', "-" * 50, '\n')
print("허브에서 프롬프트를 성공적으로 가져왔습니다.")

# 최근 커밋 해시 조회하기
prompt_id = "prompt_template_reduce"
prompt_info_reduce = client.get_prompt(prompt_id)

print("프롬프트 정보:", prompt_info_reduce)

prompt_info_reduce = client.get_prompt(prompt_id)

print("프롬프트 정보:")
for key, value in prompt_info_reduce.__dict__.items():
    print(f"{key}: {value}")

<small>

* 셀 출력 


    ```<plaintext>
    -------------------------------------------------- 

    ✅ 프롬프트 업로드 결과: https://smith.langchain.com/prompts/prompt_template_reduce/5c5d853c?organizationId=2c3342d3-1170-4ffa-86fd-f621199e0b9c

    --------------------------------------------------

    -------------------------------------------------- 

    허브에서 프롬프트를 성공적으로 가져왔습니다.
    프롬프트 정보: repo_handle='prompt_template_reduce' description='' readme='' id='102fede2-cec1-4c78-a3e1-0e6b174e9fbc' tenant_id='2c3342d3-1170-4ffa-86fd-f621199e0b9c' created_at=datetime.datetime(2025, 8, 13, 14, 11, 46, 184421) updated_at=datetime.datetime(2025, 8, 13, 14, 11, 49, 231814) is_public=False is_archived=False tags=['StringPromptTemplate'] original_repo_id=None upstream_repo_id=None owner=None full_name='prompt_template_reduce' num_likes=0 num_downloads=1 num_views=0 liked_by_auth_user=False last_commit_hash='5c5d853c9575ea9dc26be47e094ee7eaa47b329e20b011a0ef75734479133491' num_commits=1 original_repo_full_name=None upstream_repo_full_name=None
    프롬프트 정보:
    repo_handle: prompt_template_reduce
    description: 
    readme: 
    id: 102fede2-cec1-4c78-a3e1-0e6b174e9fbc
    tenant_id: 2c3342d3-1170-4ffa-86fd-f621199e0b9c
    created_at: 2025-08-13 14:11:46.184421
    updated_at: 2025-08-13 14:11:49.231814
    is_public: False
    is_archived: False
    tags: ['StringPromptTemplate']
    original_repo_id: None
    upstream_repo_id: None
    owner: None
    full_name: prompt_template_reduce
    num_likes: 0
    num_downloads: 1
    num_views: 0
    liked_by_auth_user: False
    last_commit_hash: 5c5d853c9575ea9dc26be47e094ee7eaa47b329e20b011a0ef75734479133491
    num_commits: 1
    original_repo_full_name: None
    upstream_repo_full_name: None
    ```

In [ ]:
# gemini로 LLM 초기화 및 Chain 생성하기

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langsmith import traceable

# LLM 초기화
gemini_lc = gemini_lc                                               # 위에서 정의함

# chain 구성
chain_reduce = prompt_reduce | gemini_lc | StrOutputParser()

# context_text 입력하기
# Economic News Article
context_text = """
    Central banks worldwide are intensifying their efforts to curb inflation as global inflationary pressures persist. 
    The US Federal Reserve, in a recent move, raised its benchmark interest rate by 0.25 percentage points, signaling its commitment to a restrictive monetary policy. 
    This high-interest-rate environment is likely to increase borrowing costs for businesses and dampen consumer sentiment, raising concerns about a potential economic slowdown. 
    Conversely, the rising interest rates on savings accounts are seen as a positive development for investors who prefer safe-haven assets. 
    Experts advise that central banks' future monetary policies could be flexible, depending on inflation trends and changes in the employment market, so it is crucial to monitor these indicators closely.
    """

# {doc_summaries}
result = chain_reduce.invoke({"doc_summaries": context_text})
print(result)

<small>

* 셀 출력_1 (temperature=0.7, max_tokens=4,096)(2.3s)

    ```<plaintext>
    Here are the top 10 most important insights extracted from the summaries:

    1.  **Global Inflationary Pressures Persist:** Inflation remains a significant global concern.
    2.  **Central Banks Actively Fighting Inflation:** Central banks worldwide are intensifying efforts to control inflation.
    3.  **US Federal Reserve Tightening Policy:** The US Federal Reserve has raised its benchmark interest rate by 0.25 percentage points.
    4.  **Restrictive Monetary Policy Signal:** The Fed's action signals a commitment to a restrictive monetary policy.
    5.  **Increased Borrowing Costs:** Higher interest rates will likely increase borrowing costs for businesses.
    6.  **Dampened Consumer Sentiment:** Rising interest rates are expected to negatively impact consumer sentiment.
    7.  **Risk of Economic Slowdown:** Concerns exist about a potential economic slowdown due to these factors.
    8.  **Benefit for Savers/Investors:** Rising interest rates on savings accounts are positive for investors favoring safe-haven assets.
    9.  **Monetary Policy Flexibility:** Central banks' future policies will likely be flexible.
    10. **Key Indicators to Monitor:** Inflation trends and employment market changes are crucial indicators for future policy.

    ---

    **요약된 통찰력에 대한 한국어 요약:**

    전 세계적으로 인플레이션 압력이 지속됨에 따라 중앙은행들이 인플레이션 억제에 총력을 기울이고 있습니다. 미국 연방준비제도(Fed)는 0.25%p 금리 인상을 단행하며 긴축적인 통화 정책을 시사했습니다. 이러한 고금리 환경은 기업의 차입 비용을 증가시키고 소비자 심리를 위축시켜 경기 둔화 우려를 키우고 있습니다. 반면, 저축 계좌 금리 상승은 안전 자산 선호 투자자들에게 긍정적인 신호입니다. 향후 중앙은행의 통화 정책은 인플레이션 추이와 고용 시장 변화에 따라 유연하게 결정될 가능성이 높으므로, 이러한 지표들을 면밀히 주시하는 것이 중요합니다.
    ```


In [ ]:
# context_text 입력하기
# 기술 관련 기사(한국어)
context_text = """
    최근 인공지능(AI) 기술이 의료 분야에 빠르게 적용되며 헬스케어 혁신을 주도하고 있다.
    AI는 환자의 의료 영상 분석, 질병 진단 보조, 개인 맞춤형 치료법 제안 등 다양한 영역에서 활용된다.
    특히, AI 기반 영상 진단 솔루션은 의사들이 놓칠 수 있는 미세한 병변을 정확하게 찾아내어 진단율을 크게 높이고 있다.
    한편, 빅데이터와 결합된 AI는 환자의 유전 정보, 생활 습관, 병력 등을 종합적으로 분석해 최적의 치료 경로를 제시함으로써 만성 질환 관리에도 효과적인 것으로 나타났다.
    이와 같은 기술 발전은 의료진의 업무 부담을 줄이고 환자 개개인의 삶의 질을 향상시키는 데 기여할 것으로 기대된다.
    """

# {doc_summaries}
result = chain_reduce.invoke({"doc_summaries": context_text})
print(result)

<small>


* 셀 출력_2 (temperature=0.7, max_tokens=4,096)(2.0s)

    ```<plaintext>
    ## Top 10 Most Important Insights:

    1.  **AI is rapidly transforming healthcare.**
    2.  **AI is being applied across various medical fields.**
    3.  **AI aids in medical image analysis.**
    4.  **AI assists in disease diagnosis.**
    5.  **AI proposes personalized treatment plans.**
    6.  **AI-powered diagnostic solutions improve accuracy by detecting subtle lesions.**
    7.  **AI combined with big data analyzes comprehensive patient information (genetics, lifestyle, medical history).**
    8.  **AI offers optimal treatment pathways, especially for chronic disease management.**
    9.  **AI is expected to reduce the workload of medical professionals.**
    10. **AI is anticipated to enhance the quality of life for individual patients.**

    ## 한국어 요약:

    최근 인공지능(AI) 기술이 의료 분야에 빠르게 적용되며 헬스케어 혁신을 주도하고 있습니다. AI는 의료 영상 분석, 질병 진단 보조, 개인 맞춤형 치료법 제안 등 다양한 영역에서 활용되며, 특히 영상 진단 솔루션은 의사들이 놓치기 쉬운 미세 병변을 정확히 찾아내 진단율을 높입니다. 또한, 빅데이터와 결합된 AI는 유전 정보, 생활 습관, 병력 등을 종합적으로 분석하여 최적의 치료 경로를 제시함으로써 만성 질환 관리에도 효과적인 것으로 나타났습니다. 이러한 AI 기술 발전은 의료진의 업무 부담을 줄이고 환자 개개인의 삶의 질을 향상시키는 데 크게 기여할 것으로 기대됩니다.
    ```

---

#### **4) Chain of Density Reduce Korean**

* 역할
  * 여러 **요약 리스트**를 받고, 중요한 내용을 **한국어 불릿 포인트** + 의미에 맞는 **이모지** 로 깔끔하게 재정리해 줍니다.

* 특징
  * **가장 중요한 내용이 위쪽에 오도록 정렬**
  * 불필요한 내용 배제, 핵심만 간결하게

In [22]:
# 프롬프트 생성
prompt_title = "chain-of-density-reduce-korean"

prompt_template = """You are a helpful expert in summary writing. You are given lists of summaries.
Please sum up previously summarized sentences according to the following REQUEST.
REQUEST:
1. Summarize the main points in bullet points in KOREAN.
2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
3. Use various emojis to make the summary more interesting.
4. MOST IMPORTANT points should be organized at the top of the list.
5. DO NOT include any unnecessary information.

LIST OF SUMMARIES:
{doc_summaries}

Helpful Answer: """

prompt_cdrk = PromptTemplate.from_template(prompt_template)

In [ ]:
# 허브에 올리기 
result_cdrk = client.push_prompt("prompt_template_cdrk", object=prompt_cdrk)
print('\n', "-" * 50, '\n')
print("✅ 프롬프트 업로드 결과:", result_cdrk)
print('\n', "-" * 50)

# 허브에서 가져오기 
prompt_cdrk_from_hub = client.pull_prompt("prompt_template_cdrk")
print('\n', "-" * 50, '\n')
print("허브에서 프롬프트를 성공적으로 가져왔습니다.")

# 최근 커밋 해시 조회하기
prompt_id = "prompt_template_cdrk"
prompt_info_cdrk = client.get_prompt(prompt_id)

print("프롬프트 정보:", prompt_info_cdrk)

prompt_info_cdrk = client.get_prompt(prompt_id)

print("프롬프트 정보:")
for key, value in prompt_info_cdrk.__dict__.items():
    print(f"{key}: {value}")

<small>

* 셀 출력 

    ```<plaintext>
    -------------------------------------------------- 

    ✅ 프롬프트 업로드 결과: https://smith.langchain.com/prompts/prompt_template_cdrk/45d0ab93?organizationId=2c3342d3-1170-4ffa-86fd-f621199e0b9c

    --------------------------------------------------

    -------------------------------------------------- 

    허브에서 프롬프트를 성공적으로 가져왔습니다.
    프롬프트 정보: repo_handle='prompt_template_cdrk' description='' readme='' id='374752d5-cc70-4791-a09f-45350bbf3da8' tenant_id='2c3342d3-1170-4ffa-86fd-f621199e0b9c' created_at=datetime.datetime(2025, 8, 13, 14, 33, 18, 465831) updated_at=datetime.datetime(2025, 8, 13, 14, 33, 20, 6287) is_public=False is_archived=False tags=['StringPromptTemplate'] original_repo_id=None upstream_repo_id=None owner=None full_name='prompt_template_cdrk' num_likes=0 num_downloads=1 num_views=0 liked_by_auth_user=False last_commit_hash='45d0ab9371d87694fbbc7a1804bbfdbc73c4de44c9014494538c09906488eaf7' num_commits=1 original_repo_full_name=None upstream_repo_full_name=None
    프롬프트 정보:
    repo_handle: prompt_template_cdrk
    description: 
    readme: 
    id: 374752d5-cc70-4791-a09f-45350bbf3da8
    tenant_id: 2c3342d3-1170-4ffa-86fd-f621199e0b9c
    created_at: 2025-08-13 14:33:18.465831
    updated_at: 2025-08-13 14:33:20.006287
    is_public: False
    is_archived: False
    tags: ['StringPromptTemplate']
    original_repo_id: None
    upstream_repo_id: None
    owner: None
    full_name: prompt_template_cdrk
    num_likes: 0
    num_downloads: 1
    num_views: 0
    liked_by_auth_user: False
    last_commit_hash: 45d0ab9371d87694fbbc7a1804bbfdbc73c4de44c9014494538c09906488eaf7
    num_commits: 1
    original_repo_full_name: None
    upstream_repo_full_name: None
    ```

In [ ]:
# gemini로 LLM 초기화 및 Chain 생성하기

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langsmith import traceable

# LLM 초기화
gemini_lc = gemini_lc                                               # 위에서 정의함

# chain 구성
chain_cdrk = prompt_cdrk | gemini_lc | StrOutputParser()

# context_text 입력하기
# 환경 보호 관련 신문 기사
context_text = """
    최근 기후 변화에 대한 위기의식이 고조되면서, 전 세계적으로 '탄소 중립'을 향한 노력이 가속화되고 있습니다. 
    많은 국가들이 재생 에너지 발전 비중을 늘리고, 내연기관차 판매를 단계적으로 중단하는 정책을 발표했습니다. 
    이러한 움직임은 기업에도 영향을 미쳐, ESG(환경·사회·지배구조) 경영이 중요한 가치로 떠올랐습니다. 
    하지만 탄소 중립 목표 달성은 쉽지 않은 과제입니다. 
    재생 에너지의 간헐성 문제, 에너지 저장 기술의 한계, 그리고 기존 산업 구조의 전환에 따른 경제적 부담 등이 걸림돌로 작용하고 있습니다. 
    특히 신흥국들은 경제 성장을 우선시하며 선진국과의 입장 차이를 보이고 있어 국제적인 협력이 난항을 겪기도 합니다. 
    전문가들은 기술 혁신을 통해 재생 에너지 효율을 높이고, 민간 부문의 투자를 적극적으로 유도하는 동시에, 소비자들이 친환경 제품을 선택할 수 있는 환경을 조성해야 한다고 강조합니다.
    """

# {doc_summaries}
result = chain_cdrk.invoke({"doc_summaries": context_text})
print(result)

<small>

* 셀 출력_1 (temperature=0.7, max_tokens=4,096)(1.4s)

    ```<plaintext>
    🌍 탄소 중립을 향한 전 세계적 노력 가속화

    ⚡ 재생 에너지 발전 비중 확대 및 내연기관차 판매 중단 정책 발표

    🏢 ESG 경영의 중요성 증대

    🚧 탄소 중립 목표 달성의 어려움 (재생 에너지 간헐성, 에너지 저장 기술 한계, 경제적 부담)

    🤝 신흥국과의 입장 차이로 인한 국제 협력 난항

    💡 기술 혁신, 민간 투자 유치, 친환경 소비 환경 조성을 통한 해결 노력 강조
    ```

In [ ]:
# context_text 입력하기
# Culture & Society News Article
context_text = """
    The global rise of 'digital nomadism' is reshaping the traditional concept of work and lifestyle.
    Fueled by advancements in technology and a growing preference for flexible work arrangements, professionals are now able to perform their jobs from anywhere in the world. 
    This trend has not only boosted local economies in popular destinations but has also created new challenges.
    Host countries are grappling with issues such as visa regulations, housing market pressures, and the integration of these temporary residents into local communities. 
    On the flip side, digital nomads face their own set of difficulties, including maintaining a stable work-life balance, staying connected with their home cultures, and navigating local bureaucratic hurdles.
    Experts suggest that a more structured approach, involving new policies and support systems, will be necessary to manage this evolving landscape for both the nomads and their host communities.
    """

# {doc_summaries}
result = chain_cdrk.invoke({"doc_summaries": context_text})
print(result)

<small>

* 셀 출력_2 (temperature=0.7, max_tokens=4,096)(1.5s)

    ```<plaintext>
    * 🌍 디지털 노마드 현상의 확산으로 일과 삶의 방식이 변화하고 있습니다.
    * 🚀 기술 발전과 유연한 근무 선호로 어디서든 일할 수 있게 되었습니다.
    * 📈 디지털 노마드 증가가 인기 여행지의 경제에 활력을 불어넣었지만, 새로운 과제도 야기했습니다.
    * 🛂 각국 정부는 비자 규정, 주택 시장 압박, 지역사회 통합 문제에 직면해 있습니다.
    * 🏠 디지털 노마드 또한 일과 삶의 균형 유지, 문화적 연결, 관료적 절차 등의 어려움을 겪습니다.
    * 🤝 전문가들은 이러한 변화에 대처하기 위해 정책 및 지원 시스템 마련이 필요하다고 제안합니다.
    ```

---

#### **5) Metadata Tagger**

* 역할
  * 상품 리뷰를 분석해서 고객이 언급한 디자인, 사용감, 소리, 충전, 전반적 경험 등의 **긍정/부정 포인트** 를 뽑아냄
  * **리뷰** 에서 **평점(1~5점)도 추출**

* 특징
  * 고객 감정 톤(긍정/중립/부정) 평가 포함
  * 분석 결과를 구조화된 **`JSON` 형태** 로 정리 (키워드, 평가, 개선점 등)

In [27]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

prompt_title = "metadata-tagger"

prompt_template = """Given the following product review, conduct a comprehensive analysis to extract key aspects mentioned by the customer, with a focus on evaluating the product's design and distinguishing between positive aspects and areas for improvement. 
Identify primary features or attributes of the product that the customer appreciated or highlighted, specifically looking for mentions related to the feel of the keys, sound produced by the keys, overall user experience, charging aspect, and the design of the product, etc. 
Assess the overall tone of the review (positive, neutral, or negative) based on the sentiment expressed about these attributes. 
Additionally, provide a detailed evaluation of the design, outline the positive aspects that the customer enjoyed, and note any areas of improvement or disappointment mentioned. 
Extract the customer's rating of the product on a scale of 1 to 5, as indicated at the beginning of the review. 
Summarize your findings in a structured JSON format, including an array of keywords, evaluations for design, satisfaction points, improvement areas, the assessed tone, and the numerical rating. 

INPUT:
{input}
"""

prompt_tagger = PromptTemplate.from_template(prompt_template)

In [ ]:
# 허브에 올리기 
result_tagger = client.push_prompt("prompt_template_tagger", object=prompt_tagger)
print('\n', "-" * 50, '\n')
print("✅ 프롬프트 업로드 결과:", result_tagger)
print('\n', "-" * 50)

# 허브에서 가져오기 
prompt_tagger_from_hub = client.pull_prompt("prompt_template_tagger")
print('\n', "-" * 50, '\n')
print("허브에서 프롬프트를 성공적으로 가져왔습니다.")

# 최근 커밋 해시 조회하기
prompt_id = "prompt_template_tagger"
prompt_info_tagger = client.get_prompt(prompt_id)

print("프롬프트 정보:", prompt_info_tagger)

prompt_info_tagger = client.get_prompt(prompt_id)

print("프롬프트 정보:")
for key, value in prompt_info_tagger.__dict__.items():
    print(f"{key}: {value}")

<small>

* 셀 출력

    ```<plaintext>
    -------------------------------------------------- 

    ✅ 프롬프트 업로드 결과: https://smith.langchain.com/prompts/prompt_template_tagger/b66c04b7?organizationId=2c3342d3-1170-4ffa-86fd-f621199e0b9c

    --------------------------------------------------

    -------------------------------------------------- 

    허브에서 프롬프트를 성공적으로 가져왔습니다.
    프롬프트 정보: repo_handle='prompt_template_tagger' description='' readme='' id='1c22512f-b554-47d4-8a2f-056588896a18' tenant_id='2c3342d3-1170-4ffa-86fd-f621199e0b9c' created_at=datetime.datetime(2025, 8, 13, 14, 52, 10, 627018) updated_at=datetime.datetime(2025, 8, 13, 14, 52, 12, 270861) is_public=False is_archived=False tags=['StringPromptTemplate'] original_repo_id=None upstream_repo_id=None owner=None full_name='prompt_template_tagger' num_likes=0 num_downloads=1 num_views=0 liked_by_auth_user=False last_commit_hash='b66c04b78d310143945a9c5679551137fc66b72d3d712357e7773ab6e06baf53' num_commits=1 original_repo_full_name=None upstream_repo_full_name=None
    프롬프트 정보:
    repo_handle: prompt_template_tagger
    description: 
    readme: 
    id: 1c22512f-b554-47d4-8a2f-056588896a18
    tenant_id: 2c3342d3-1170-4ffa-86fd-f621199e0b9c
    created_at: 2025-08-13 14:52:10.627018
    updated_at: 2025-08-13 14:52:12.270861
    is_public: False
    is_archived: False
    tags: ['StringPromptTemplate']
    original_repo_id: None
    upstream_repo_id: None
    owner: None
    full_name: prompt_template_tagger
    num_likes: 0
    num_downloads: 1
    num_views: 0
    liked_by_auth_user: False
    last_commit_hash: b66c04b78d310143945a9c5679551137fc66b72d3d712357e7773ab6e06baf53
    num_commits: 1
    original_repo_full_name: None
    upstream_repo_full_name: None
    ```

In [ ]:
# gemini로 LLM 초기화 및 Chain 생성하기

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langsmith import traceable

# LLM 초기화
gemini_lc = gemini_lc                                               # 위에서 정의함

# chain 구성
chain_taggger = prompt_tagger | gemini_lc | StrOutputParser()

# context_text 입력하기
# 긍정 리뷰 예시
context_text = """
    I had a truly fantastic experience at 'The Daily Grind' cafe this past weekend. 
    The coffee was perfectly brewed, with a rich, smooth flavor that was absolutely delightful. 
    What really stood out was the incredible service. 
    The staff were attentive, friendly, and went out of their way to make sure we were comfortable. 
    The ambiance was also wonderful; 
    the soft music and cozy decor made it the perfect place to relax and catch up with a friend. 
    I highly recommend this place for anyone looking for great coffee and a welcoming atmosphere. 
    I will definitely be a regular here.
    """

# {doc_summaries}
result = chain_taggger.invoke({"doc_summaries": context_text})
print(result)